In [13]:
import numpy as np
import zlib
import zipfile
import gzip
import time

In [25]:
def distance(a, b):
    time_1=time.time()
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n, m)) space
        a, b = b, a
        n, m = m, n

    current_row = range(n + 1)  # Keep current and previous row, not entire matrix
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)
    print(time.time()-time_1)
    return current_row[n]

In [4]:
words={}
words_l=[]
with open('lenta_words.txt', encoding='utf-8') as input_file: 
    for i in (input_file):
        i=i.strip()
        if i in words:
            value = words[i]
            words[i]=value+1
        else:
            words_l.append(i)
            words[i]=1

In [5]:
def distances(word, voc):
    dist=[]
    for w in voc:
        dist.append(distance(word,w))
    dist=np.array(dist)
    ind = np.where(dist == dist.min())[0]
    freq=int(0)
    for i in ind:
        if freq<words[words_l[i]]:
            freq=words[words_l[i]]
            word=words_l[i]
    return word

In [20]:
z=["путн", "оцинил","роботу","новвых", "самалетав", "и","виртолтов", "сирийи"]
correct_words=[]
for i in z:
    correct_words.append(distances(i,words))

In [32]:
for i in range(len(z)):
    print(distance(z[i],correct_words[i]))
    print(distance_2(z[i], correct_words[i]))

0.0
1
0.0
1
0.0
1
0.0
1
0.0
0
0.0
0
0.0
1
0.0
1
0.0
2
0.0
2
0.0
0
0.0
0
0.0
2
0.0
2
0.0
1
0.0
1


In [27]:
def distance_2(a, b):
    # Calculates the Levenshtein distance between a and b
    time_1=time.time()
    n, m = len(a), len(b)
    inverse = False
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a, b = b, a
        n, m = m, n
        inverse = True

    max_distance = m + n  # to avoid index out of range in possible_action creation, fictive column
    current_row = list(range(n + 1)) + [max_distance]  # Keep current and previous row, not entire matrix

    matrix = np.array([current_row])
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n + [max_distance]
        for j in range(1, n + 1):
            # add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            add, delete, change = previous_row[j] + 1, \
                                  current_row[j - 1] + 1, \
                                  previous_row[j - 1] + int(a[j - 1] != b[i - 1])
            # if a[j - 1] != b[i - 1]:
            #     change += 1
            current_row[j] = min(add, delete, change)

        matrix = np.vstack((matrix, [current_row]))

    lev_distance = current_row[n]
    # print(matrix)

    if inverse:
        matrix = matrix.T
        a, b = b, a
        n, m = m, n
    print(time.time()-time_1)
    return lev_distance

In [37]:
import pickle


def read(file_name):
    with open(file_name, 'r',  encoding='utf-8') as f:
        content = f.readlines()
        content = [l.strip('\n') for l in content]

    return content


def flatten_list(lol):
    return [item for sublist in lol for item in sublist]


def flatten_dictionary(dictionary):
    values = []
    for d in dictionary.values():
        for v in d.values():
            values.append(v)
    return values


def dump(obj, file_name):
    with open(file_name, 'w',  encoding='utf-8') as f:
        pickle.dump(obj, f)


def load(file_name):
    with open(file_name, 'r',  encoding='utf-8') as f:
        return pickle.load(f)

In [40]:
class ErrorModel:

    def __init__(self):
        self.insertions = defaultdict(int)
        self.deletions = defaultdict(int)
        self.replacements = defaultdict(lambda: defaultdict(int))

    def update_statistics(self, given, fixed):
        operations = Levenshtein.opcodes(given, fixed)
        for op in operations:
            name, i1, i2, j1, j2 = op[0], op[1], op[2], op[3], op[4]
            if name == 'insert':
                for c in fixed[j1:j2]:
                    self.insertions[c] += 1
            if name == 'delete':
                for c in given[i1:i2]:
                    self.deletions[c] += 1
            if name == 'replace':
                for c1, c2 in zip(given[i1:i2], fixed[j1:j2]):
                    self.replacements[c1][c2] += 1

    def calculate_weights(self):
        self.calculate_replacement_weights()
        self.deletion_weights = ErrorModel.calculate_list_weights(
            self.deletions)
        self.insertion_weights = ErrorModel.calculate_list_weights(
            self.insertions)

    @staticmethod
    def calculate_list_weights(dictionary):
        list_frequencies_to_weights, default_weight = ErrorModel.prepare_weights(
            dictionary.values())
        list_weights = defaultdict(lambda: default_weight)
        for k in dictionary.keys():
            list_weights[k] = list_frequencies_to_weights[dictionary[k]]
        return list_weights

    def calculate_replacement_weights(self):

        replacement_frequencies_to_weights, default_weight = ErrorModel.prepare_weights(
            (flatten_dictionary(self.replacements)))

        self.replacement_weights = defaultdict(
            lambda: defaultdict(lambda: default_weight))
        for k1, v in self.replacements.items():
            for k2 in v.keys():
                self.replacement_weights[k1][
                    k2] = replacement_frequencies_to_weights[self.replacements[k1][k2]]

    @staticmethod
    def prepare_weights(values):
        list_frequencies = np.sort(np.array(values))[::-1]
        list_weights = np.log1p(list_frequencies).astype(float)[::-1]
        list_frequencies_to_weights = {}
        for i in range(len(list_frequencies)):
            list_frequencies_to_weights[list_frequencies[i]] = list_weights[i]
        default_weight = np.max(list_weights)
        return list_frequencies_to_weights, default_weight

In [42]:
import dill  
import pandas as pd
from re import escape
from string import punctuation
from collections import Counter
from collections import defaultdict

In [43]:
queries = read('queries_all.txt')
queries = [q.split('\t') for q in queries]
# queries_lod = []
original_queries = []
fixed_queries = []
for q in queries:
    if len(q) == 2:
        original_queries.append(q[0])
        fixed_queries.append(q[1])
        # queries_lod.append({'query': q[0], 'fixed': q[1], 'needs_fix': True})
    else:
        original_queries.append(q[0])
        fixed_queries.append(q[0])
        # queries_lod.append({'query': q[0], 'fixed': q[0], 'needs_fix': False})
# queries_df = pd.DataFrame(queries_lod)

# SPLIT

punctuation = escape(punctuation)

# fixed_queries_to_words =
# queries_df['fixed'].str.decode('utf-8').replace('[' + punctuation +']',
# '', regex=True).str.split()
fixed_queries_to_words = pd.Series(fixed_queries).replace('[' + punctuation + ']', '', regex=True).str.split()
fixed_words = flatten_list(fixed_queries_to_words)
frequency_dictionary_of_fixed_words = Counter(fixed_words)

# original_queries_to_words =
# queries_df['query'].str.decode('utf-8').replace('[' + punctuation +']',
# '', regex=True).str.split()
original_queries_to_words = pd.Series(original_queries).replace('[' + punctuation + ']', '', regex=True).str.split()
original_words = flatten_list(original_queries_to_words)
# frequency_dictionary_of_original_words = Counter(original_words)

# CALCULATE ERROR MODEL

error_model = ErrorModel()

for original, fixed in zip(original_queries_to_words, fixed_queries_to_words):
    number_of_words = min(len(original), len(fixed))
    for i in range(number_of_words):
        error_model.update_statistics(original[i], fixed[i])

error_model.calculate_weights()

NameError: name 'Levenshtein' is not defined